In [ ]:
import pandas as pd
import numpy as np
import os
import json
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install openai

In [ ]:
import openai

In [ ]:
from openai import OpenAI
prompt="hh"
client = OpenAI(
    api_key="",
    base_url="https://api.aimlapi.com",
)

response = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)

print(response)

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_final.csv")

In [ ]:
import time
gens = []
answers = []
cnt = 0
text_list=[]
for i in range(0, len(dataset)-1, 2):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]

    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"
    text_list.append(text1)
    text_list.append(text2)
    prompt = f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\nAnswer in the format that 1.Answer for the first question 2. 1.Answer for the second question\nA:"
    gens.append(
        client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
    )

    answers.append([ans1, ans2])

    print("="*20)
    print(i, "\n")
    print(prompt)
    print(gens)
    cnt += 1
    # if(i>5):
    #   break

print(cnt)

df = pd.DataFrame(text_list,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_mis.csv', index=False)
results = [gen.choices[0].message.content.strip() for gen in gens]



In [ ]:
# import math
# if math.isnan(dataset["mod"][i]):
#     print(1)
df = pd.DataFrame(results,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_mis_answer.csv', index=False)

In [ ]:
[res.split("\n") for res in  results]

In [ ]:
for res in  results:
  print(res)
  print("="*20)

In [ ]:

answer = 'In the first sentence, "he" refers to Tom. In the second sentence, "he" refers to Ray.'
gold = ["Tom", "Ray"]

def loosely_match_results(a_, ans_):
    if a_.lower() in ans_[0].lower():
        return 0
    elif a_.lower().replace("the", "").strip() in ans_[0].lower():
        return 0
    elif a_.lower().replace("a", "").strip() in ans_[0].lower():
        return 0
    elif a_.lower() in ans_[1].lower():
        return 1
    elif a_.lower().replace("the", "").strip() in ans_[1].lower():
        return 1
    elif a_.lower().replace("a", "").strip() in ans_[1].lower():
        return 1
    else:
        return -1


def parse_answer_1(pred):
    # print(pred)
    if "1." not in pred:
        # print(pred)
        print("not follow the instruction")
        return "",""
    if "2." not in pred:
        # print(pred)
        print("not follow the instruction")
        return "",""
    pred=pred[pred.rfind("1."):].strip()


    ans1, ans2 = pred.split("2. ")
    ans1, ans2 =ans1.split("\n")[0], ans2.split("\n")[0]
    ans1, ans2 = ans1.strip(), ans2.strip()
    ans1 = ans1.replace("1.", "")
    if "refers to" in ans1:
        idx = ans1.find("refers to") + len("refers to")
        ans1 = ans1[idx:].replace(".", "")
    elif "refer to" in ans1:
        idx = ans1.find("refer to") + len("refer to")
        ans1 = ans1[idx:].replace(".", "")
    else:
        ans1 = ans1



    ans2 = ans2.replace("2.", "")
    if "refers to" in ans2:
        idx = ans2.find("refers to") + len("refers to")
        ans2 = ans2[idx:].replace(".", "")
    elif "refer to" in ans2:
        idx = ans2.find("refer to") + len("refer to")
        ans2 = ans2[idx:].replace(".", "")
    else:
        ans2 = ans2

    return ans1.replace('"', '').strip(), ans2.replace('"', '').strip()



In [ ]:
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results, answers):
    print("*"*20)

    a1, a2 = parse_answer_1(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)

    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    if pred_0 == -1 or pred_1 == -1:
        print(result)
        print("wrong parsing", a1, a2, answer)
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))
#     print(pred_0 == 0 and pred_1 == 1, "|", a1, "|", a2, "|", answer)
print("em", em/len(results)/2, "acc", correct_cnt/len(results)/2)

df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_mis.csv', index=False)

# one-shot

In [ ]:
import time
gens_oneshot = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"

    examplar = "Q: Compare the two sentences and answer the questions\n1. The fish ate the worm. It was hungry. What does \"it\" refer to?\n2. The fish ate the worm. It was tasty. What does \"it\" refer to?\nSelect from [\"The fish\", \"The worm\"]\nA: 1. The fish. 2. The worm"

    prompt = examplar + "\n\n" + f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\nA: "

    gens_oneshot.append(
        client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
        )
    answers.append([ans1, ans2])
    time.sleep(0.1)
#     print("="*20)
#     print(i, "\n")
#     print(prompt)
    cnt += 1





In [ ]:
results_oneshot = [gen.choices[0].message.content.strip() for gen in gens_oneshot]
print(results_oneshot)
df = pd.DataFrame(results_oneshot,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_one_shot_mis_answer.csv', index=False)

In [ ]:
def parse_answer_oneshot(pred):
    if "1." not in pred:
        print(pred)
        print("not follow the instruction")
        return "",""
    if "2." not in pred:
        print(pred)
        print("not follow the instruction")
        return "",""
    pred=pred[pred.rfind("1."):].strip()


    ans1, ans2 = pred.split("2. ")
    ans1, ans2 =ans1.split("\n")[0], ans2.split("\n")[0]
    ans1, ans2 =ans1.split("(")[0], ans2.split("(")[0]
    ans1, ans2 = ans1.strip(), ans2.strip()
    ans1 = ans1.replace("1.", "")
    if "refers to" in ans1:
        idx = ans1.find("refers to") + len("refers to")
        ans1 = ans1[idx:].replace(".", "")
    elif "refer to" in ans1:
        idx = ans1.find("refer to") + len("refer to")
        ans1 = ans1[idx:].replace(".", "")
    else:
        ans1 = ans1



    ans2 = ans2.replace("2.", "")
    if "refers to" in ans2:
        idx = ans2.find("refers to") + len("refers to")
        ans2 = ans2[idx:].replace(".", "")
    elif "refer to" in ans2:
        idx = ans2.find("refer to") + len("refer to")
        ans2 = ans2[idx:].replace(".", "")
    else:
        ans2 = ans2

    if ans1.endswith("."):
        ans1 = ans1.replace(".", "")
    if ans2.endswith("."):
        ans2 = ans2.replace(".", "")
    return ans1.replace('"', '').strip(), ans2.replace('"', '').strip()

In [ ]:
# oneshot: fish and worm case
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results_oneshot, answers):
    print("*"*20)
    a1, a2 = parse_answer_oneshot(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)
    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    # if pred_0 == -1 or pred_1 == -1:
    #     print(result)
    #     print("wrong parsing", a1, a2)
    #     print(answer)
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))

df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_one_shot_result_mis.csv', index=False)
print("em", em/len(results)/2, "acc", correct_cnt/len(results)/2)

Cot-one shot

In [ ]:
import time
gens_oneshot = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"

    examplar = "Q: Compare the two sentences and answer the questions\n\
1. The fish ate the worm, it was tasty. What does \"it\" refer to?\n\
2. The fish ate the worm, it was hungry. What does \"it\" refer to?\n\
Select from [\"fish\", \"worm\"]\n\n\
In the first sentence, the worm is the main object that was eaten, the one that is eaten should be considered as tasty. In the second sentence, the fish was the one eating so it must be hungry. Thus the answer is:\n\
A: 1. worm 2. fish"

    prompt = examplar + "\n\n" + f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\nA: "
    # gens_oneshot.append(openai.Completion.create(
    #               model="text-davinci-003",
    #               prompt=prompt,
    #               max_tokens=256,
    #               temperature=0
    # ))
    gens_oneshot.append(
        client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
        )
    answers.append([ans1, ans2])
    time.sleep(0.1)
    print("="*20)
    print(i, "\n")
    print(prompt)
    cnt += 1





In [ ]:
results_oneshot = [gen.choices[0].message.content.strip() for gen in gens_oneshot]
print(results_oneshot)
df = pd.DataFrame(results_oneshot,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_cot_one_shot_mis_answer.csv', index=False)

In [ ]:
# oneshot: fish and worm case
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results_oneshot, answers):
    a1, a2 = parse_answer_oneshot(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)
    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    if pred_0 == -1 or pred_1 == -1:
        print("*"*100)
        print(result)
        print("wrong parsing", a1, a2, answer)
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))
#     print(pred_0 == 0 and pred_1 == 1, "|", a1, "|", a2, "|", answer)
df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_cot_one_shot_result_mis.csv', index=False)
print("em", em/len(results_oneshot)/2, "acc", correct_cnt/len(results_oneshot)/2)

CoT-Let's think step by step

In [ ]:
import time
gens_oneshot = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"

    # examplar = "Q: Compare the two sentences and answer the questions\n1. The fish ate the worm. It was hungry. What does \"it\" refer to?\n2. The fish ate the worm. It was tasty. What does \"it\" refer to?\nSelect from [\"The fish\", \"The worm\"]\nA: 1. The fish. 2. The worm"

    prompt =  f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\nAnswer in the format that 1.Answer for the first question 2. 1.Answer for the second question\nlet's think step by step\nA: "
    # gens_oneshot.append(openai.Completion.create(
    #               model="text-davinci-003",
    #               prompt=prompt,
    #               max_tokens=256,
    #               temperature=0
    # ))
    gens_oneshot.append(
        client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
        )
    answers.append([ans1, ans2])
    time.sleep(0.1)
#     print("="*20)
#     print(i, "\n")
#     print(prompt)
    cnt += 1

In [ ]:
results_oneshot = [gen.choices[0].message.content.strip() for gen in gens_oneshot]
print(results_oneshot)
df = pd.DataFrame(results_oneshot,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_cot_mis_answer.csv', index=False)

In [ ]:
# oneshot: fish and worm case
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results_oneshot, answers):
    print("*"*20)
    a1, a2 = parse_answer_oneshot(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)
    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    # if pred_0 == -1 or pred_1 == -1:
    #     print(result)
    #     print("wrong parsing")
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))

df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_step_result_mis.csv', index=False)
print("em", em/len(results_oneshot)/2, "acc", correct_cnt/len(results_oneshot)/2)

# CoT conceptualization

In [ ]:
import time
from tqdm import tqdm
cot_concept_prompt = "Q: Compare the two sentences and answer the questions\n\
1. The tasty fish ate the worm, it was tasty. What does \"it\" refer to?\n\
2. The tasty fish ate the worm, it was hungry. What does \"it\" refer to?\n\
Select from [\"tasty fish\", \"worm\"]\n\n\
Conceptualization: Fish can be conceptualized as a predator, and worm can be conceptualized as a prey.\n\
The question can be conceptualized as:\n\
1. The predator ate the prey, it was tasty. What does \"it\" refer to?\n\
2. The predator ate the prey, it was hungry. What does \"it\" refer to?\n\
Select from [\"prey\", \"predator\"]\n\
Because the subject of \"ate\" should be hungry and the object should be tasty, so:\n\
Answer: 1. prey. 2. predator\n\n\
Conclusion: As worm is a prey, and fish is a predator in the context,\n\
A: Thus the answer is:\n\
1. worm 2. fish"

gens_cot_concept = []
answers = []
cnt = 0
for i in tqdm(range(0, len(dataset)-1, 2)):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"

    prompt = cot_concept_prompt + "\n\n" + f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\nA: "
    gens_cot_concept.append(
      client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
      )
    )
    answers.append([ans1, ans2])
    time.sleep(0.1)
#     print(prompt)
    cnt += 1


results_cot_concept_direct = [gen.choices[0].message.content.strip() for gen in gens_cot_concept]
print(results_cot_concept_direct)

In [ ]:
df = pd.DataFrame(results,columns=['text'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_result_AoT_mis_answer.csv', index=False)

In [ ]:
# cot: direct prompt answer
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results_cot_concept_direct, answers):
    print("*"*10)
    a1, a2 = parse_answer_oneshot(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)
    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    if pred_0 == -1 or pred_1 == -1:
        print(result)
        print("wrong parsing", a1, a2, answer)
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))
#     print(pred_0 == 0 and pred_1 == 1, "|", a1, "|", a2, "|", answer)
df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_AoT_result_mis.csv', index=False)
print("em", em/len(results)/2, "acc", correct_cnt/len(results)/2)

# WinoWhy style exampler

In [ ]:
# winowhy

cot_winowhy_prompt = "Q: Compare the two sentences and answer the questions\n\
1. The tasty fish ate the worm, it was tasty. What does \"it\" refer to?\n\
2. The tasty fish ate the worm, it was hungry. What does \"it\" refer to?\n\
Select from [\"tasty fish\", \"worm\"]\n\n\
In the first sentence, the worm is the main object that the very (ate) refers to so the answer is worm. In the second sentence, the fish was the one eating so it must be hungry. Thus the answer is:\n\
A: 1. worm 2. fish"

### ask to generate all conceptualization

import time
from tqdm import tqdm


gens_winowhy = []
answers = []
cnt = 0
for i in tqdm(range(0, len(dataset)-1, 2)):
    if i >= 254:
        i += 1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ans[0], ans[1]
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"

    prompt = cot_winowhy_prompt + "\n\n" + f"Q: Compare the two sentences and answer the questions\n1. {text1 + get_question(pronoun)}\n2. {text2 + get_question(pronoun)}\nSelect from [\"{ans1}\", \"{ans2}\"]\n"
    gens_winowhy.append(
        client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    )
        )
    answers.append([ans1, ans2])

    cnt += 1

results_winowhy = [gen.choices[0].message.content.strip() for gen in gens_winowhy]




In [ ]:
# cot: direct prompt answer
parsed_results = []
em = 0
correct_cnt = 0
predictions = []
for result, answer in zip(results_winowhy, answers):
    if result.startswith("A:"):
        result = result[2:]
    if result.find("\nA:") != -1:
        result = result[result.find("\nA:")+len("\nA:"):]

    print(result)
    a1, a2 = parse_answer_oneshot(result)
    pred_0 = loosely_match_results(a1, answer)
    pred_1 = loosely_match_results(a2, answer)
    if pred_0 == 0 and pred_1 == 1:
        em += 2
    correct_cnt += int((pred_0 == 0)) + int((pred_1 == 1))
    if pred_0 == -1 or pred_1 == -1:
        print("wrong parsing", a1, a2, answer)
    predictions.append(int(pred_0 == 0))
    predictions.append(int(pred_1 == 1))
#     print(pred_0 == 0 and pred_1 == 1, "|", a1, "|", a2, "|", answer)
df = pd.DataFrame(predictions,columns=['result'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/text_origin_tested_win_result_mis.csv', index=False)
print("em", em/len(results_winowhy)/2, "acc", correct_cnt/len(results_winowhy)/2)